# 프롬프트 (`Prompt`)

- LLM한테 주는 입력(지시, 맥락, 기록)
    - 지시: '~~해줘'
    - 맥락: Context - 현재 지시를 위해 제공하는 추가 정보
    - 기억: Memory - 지금까지 했던 대화 내용

In [2]:
from dotenv import load_dotenv

load_dotenv()

True

## `PromptTemplate`

- 단순히 1회성 명령을 내릴 때 사용

In [5]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

llm = ChatOpenAI(model='gpt-4.1-nano')

In [ ]:
# 추후에 체인.invoke에서 {}내부에 들어갈 말을 채워줘야 함.
template = '{country}의 수도가 어디인가요?'

# .from_template 메서드로 프롬프트 만들기
prompt = PromptTemplate.from_template(template)

# {}를 채우는 방법 (우리는 결국 chain.invoke로 쓰게 됨)
prompt.format(country='대한민국')

'대한민국의 수도가 어디인가요?'

In [6]:
chain = prompt | llm

chain.invoke({'country': '대한민국'}).content

'대한민국의 수도는 서울입니다.'

### Partial Variable (부분 변수)

- 프롬프트에서 매개변수 기본값 사용하기

In [7]:
template = '{c1}과 {c2}의 수도는 각각 어디인가요?'

prompt = PromptTemplate(
    template=template,
    input_variables=['c1'],
    partial_variables={
        'c2': '미국'
    }
)

prompt.format(c1='한국')
# prompt.format(c2='캐나다') 

'한국과 미국의 수도는 각각 어디인가요?'

In [8]:
chain = prompt | llm
chain.invoke({'c1': 'kor', 'c2': 'us'}).content

'한국(대한민국, KOR)의 수도는 서울입니다.  \n미국(US)의 수도는 워싱턴 D.C.입니다.'

In [9]:
from datetime import datetime

def get_today():
    return datetime.now().strftime('%b %d')

prompt = PromptTemplate(
    template='오늘 날짜는 {today}입니다. 오늘 생일인 유명인 {n}명을 생년월일과 함께 나열해 주세요',
    input_variables=['n'],
    partial_variables={
        'today': get_today
    }
)

prompt.format(n=3)

'오늘 날짜는 Sep 01입니다. 오늘 생일인 유명인 3명을 생년월일과 함께 나열해 주세요'

In [10]:
llm = ChatOpenAI(model='gpt-5-nano')

chain = prompt | llm
# 오늘 날짜 기준
print(chain.invoke({'n': 3}).content)
print('===')
print(chain.invoke({'n': 3, 'today': 'Jan 2'}).content)

오늘(9월 1일) 생일인 유명인 중 확인된 두 명과 생년월일은 아래와 같습니다.

- Zendaya — 1996년 9월 1일
- Gloria Estefan — 1957년 9월 1일

세 번째 인물은 인터넷에서 확인이 더 필요합니다. 정확한 정보를 위해 웹 검색으로 한 분 더 찾아서 함께 적어드릴까요?
===
오늘이 1월 2일이라는 점을 반영해, 1월 2일에 생일인 유명인으로 널리 알려진 사람부터 하나 알려드리겠습니다.

- Isaac Asimov — 1920년 1월 2일

나머지 두 명은 정확한 생년월일 확인이 필요합니다. 원하시면 신뢰 가능한 소스에서 찾아 세 사람 모두 생년월일과 함께 확정해서 드리겠습니다. 특정 분야(영화/음악 등)로 범위를 좁히면 더 빨리 찾아드릴 수 있어요. 계속 진행할까요?


## `ChatPromptTemplate`

- 채팅을 주고받는 템플릿 생성용
- 대화 목록을 LLM에게 주입
- 하나의 Chat은 `role`과 `message`로 구성됨

In [11]:
from langchain_core.prompts import ChatPromptTemplate

chat_prompt = ChatPromptTemplate.from_template('{country}의 수도는 어디?')
chat_prompt.format(country='한국')

'Human: 한국의 수도는 어디?'

In [12]:
chat_template = ChatPromptTemplate.from_messages(
    [
        # role - message
        ('system', '당신은 친절한 AI어시스트. 이름은 {name} 야.'),
        ('human', '반가워!'),
        ('ai', '무엇을 도와드릴까요?'),
        ('human', '{user_input}')
    ]
)

# format vs format_messages
messages_str = chat_template.format(name='gaida', user_input='이름이 뭐니?')
messages_cls = chat_template.format_messages(name='gaida', user_input='이름이 뭐니?')
print(messages_str)
print(messages_cls)

System: 당신은 친절한 AI어시스트. 이름은 gaida 야.
Human: 반가워!
AI: 무엇을 도와드릴까요?
Human: 이름이 뭐니?
[SystemMessage(content='당신은 친절한 AI어시스트. 이름은 gaida 야.', additional_kwargs={}, response_metadata={}), HumanMessage(content='반가워!', additional_kwargs={}, response_metadata={}), AIMessage(content='무엇을 도와드릴까요?', additional_kwargs={}, response_metadata={}), HumanMessage(content='이름이 뭐니?', additional_kwargs={}, response_metadata={})]


In [13]:
# 한덩어리 텍스트
llm.invoke(messages_str).content
# 실제 대화 내역으로 인지 (Langsmith 가서 확인)
llm.invoke(messages_cls).content

'제 이름은 gaida예요. 반가워요! 필요한 게 있으면 말씀해 주세요.'

In [14]:
chain = chat_template | llm | StrOutputParser()

chain.invoke({'name': '최은비', 'user_input': '내 이름은 배성우. 너와 나의 이름점을 봐줘'})

'배성우님, 반가워요! 이름 점은 재미로 보는 거니까 가볍게 한 번 계산해볼게요. 이번에는 초성 숫자 합 방식으로 간단한 궁합을 보겠습니다. 규칙은 이렇게 정했어요: 각 초성에 1부터 19까지 숫자를 매겨 합산해요.\n\n초성 매핑 예시\n- ㄱ=1, ㄲ=2, ㄴ=3, ㄷ=4, ㄸ=5, ㄹ=6, ㅁ=7, ㅂ=8, ㅃ=9, ㅅ=10, ㅆ=11, ㅇ=12, ㅈ=13, ㅉ=14, ㅊ=15, ㅋ=16, ㅌ=17, ㅍ=18, ㅎ=19\n\n결과\n- 배성우: 배(ㅂ=8) + 성(ㅅ=10) + 우(ㅇ=12) = 합계 30\n- 최은비: 최(ㅊ=15) + 은(ㅇ=12) + 비(ㅂ=8) = 합계 35\n\n궁합 점수\n- 작은 수를 큰 수로 나눈 값의 비율로 간단히 표현: 30 / 35 ≈ 0.857 → 약 85.7% → 대략 86%\n\n해석\n- 합이 비슷하고 서로의 리듬이 잘 맞는 편이에요. 배성우님의 추진력과 최은비님의 안정감이 서로 보완해 주어 함께 할 때 시너지가 날 가능성이 큽니다. 단, 이름 궁합은 재미로 보는 것이니 너무 진지하게 받아들이지 마시고 가볍게 즐겨주세요.\n\n다른 방식으로도 더 해볼까요?\n- 글자 의미/음운 조화로 보기\n- 한자 이름의 음양오행 관점으로 해석\n- 서로의 성격 포인트를 추정하는 비유적 해석\n\n원하시는 방식이 있으면 알려 주세요. 또 다른 방식으로도 재미있게 더 봐드릴게요!'

## 프롬프팅
내가 원하는 답변을 구구절절 설명 X -> 예시를 주고 답변을 형태를 유도

- Zero Shot prompting : 예시 없이 질문만 -> 바로 답변
- One-shot prompting : 예시 1개 + 질문 -> 모델이 예시를 모방해서 답변
- **Few-shot prompting** : 예시 여러개 + 질문 -> 예시들의 패턴을 일반화해서 답변

In [16]:
from langchain_core.prompts.few_shot import FewShotPromptTemplate
from langchain_core.output_parsers import StrOutputParser

llm = ChatOpenAI(model='gpt-4.1-nano')

examples = [
    {
        "question": "스티브 잡스와 아인슈타인 중 누가 더 오래 살았나요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 스티브 잡스는 몇 살에 사망했나요?
중간 답변: 스티브 잡스는 56세에 사망했습니다.
추가 질문: 아인슈타인은 몇 살에 사망했나요?
중간 답변: 아인슈타인은 76세에 사망했습니다.
최종 답변은: 아인슈타인
""",
    },
    {
        "question": "네이버의 창립자는 언제 태어났나요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 네이버의 창립자는 누구인가요?
중간 답변: 네이버는 이해진에 의해 창립되었습니다.
추가 질문: 이해진은 언제 태어났나요?
중간 답변: 이해진은 1967년 6월 22일에 태어났습니다.
최종 답변은: 1967년 6월 22일
""",
    },
    {
        "question": "율곡 이이의 어머니가 태어난 해의 통치하던 왕은 누구인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 율곡 이이의 어머니는 누구인가요?
중간 답변: 율곡 이이의 어머니는 신사임당입니다.
추가 질문: 신사임당은 언제 태어났나요?
중간 답변: 신사임당은 1504년에 태어났습니다.
추가 질문: 1504년에 조선을 통치한 왕은 누구인가요?
중간 답변: 1504년에 조선을 통치한 왕은 연산군입니다.
최종 답변은: 연산군
""",
    },
    {
        "question": "올드보이와 기생충의 감독이 같은 나라 출신인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 올드보이의 감독은 누구인가요?
중간 답변: 올드보이의 감독은 박찬욱입니다.
추가 질문: 박찬욱은 어느 나라 출신인가요?
중간 답변: 박찬욱은 대한민국 출신입니다.
추가 질문: 기생충의 감독은 누구인가요?
중간 답변: 기생충의 감독은 봉준호입니다.
추가 질문: 봉준호는 어느 나라 출신인가요?
중간 답변: 봉준호는 대한민국 출신입니다.
최종 답변은: 예
""",
    },
]

example_prompt = PromptTemplate.from_template(
    "Question:\n{question}\nAnswer:\n{answer}"
)

prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Question:\n{question}\nAnswer:",
    input_variables=["question"],
)

question = "Google이 창립된 연도에 Bill Gates의 나이는 몇 살인가요?"

chain = prompt | llm | StrOutputParser()

res = chain.invoke({'question': question})

print(res)

이 질문에 추가 질문이 필요한가요: 예.  
추가 질문: Google이 창립된 연도는 언제인가요?  
중간 답변: Google은 1998년에 창립되었습니다.  
추가 질문: Bill Gates는 1998년 몇 살이었나요?  
중간 답변: Bill Gates는 1998년 43살이었습니다.  
최종 답변은: 43살입니다.


## 다른 좋은 프롬프트 활용하기 (langchain-hub)

- 다양한 사용자들이 업로드한 프롬프트를 받아서 활용

In [17]:
from langchain import hub

prompt = hub.pull('hwchase17/react')
print(prompt.template)

Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [18]:
prompt = PromptTemplate.from_template(
    '내 첫 프롬프트 Push!'
)

hub.push('gaida1st/prompt-hub-test', prompt)

LangSmithUserError: Cannot create a prompt for another tenant.
Current tenant: None,
Requested tenant: gaida1st